# Convolutional Nueral Networks
This week we will implement the backward propogation for different layers in a convloutional neural networks in. All the coding task will take place in "**cnn/layers.py**". We provide the implmentation for forward propogation, and the skeleton & instruction for the backward propogation.

* Task 0: Implement the backward pass for the fully connected layer
* Task 1: Implement the backward pass for the Convolutional layer with forloop
* Task 3: Implement the backward pass for the Max pooling layer with forloop
* Task 3: Implement backward pass for the Relu layer
* Task 4: Try fast implement for Convolution and Max pooling layers. Make sure you understand how it works.

** FIRST: create a new folder called "notebooks/cnn", unzip these files into it. **

In [1]:
# As usual, Here is a bit of setup
import numpy as np
import matplotlib.pyplot as plt
from notebooks.utils.data_utils import load_CIFAR10
from scipy.misc import imread, imresize
from notebooks.utils.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from cnn.layers import *
from cnn.fast_layers import *
from time import time

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def get_CIFAR10_data(num_training=4900, num_validation=100, num_test=100):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    print('Loading CIFAR10 ...')
    # Load the raw CIFAR-10 data
    cifar10_dir = '../opt/data/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
        
    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    
    # Transpose so that channels come first
    X_train = X_train.transpose(0, 3, 1, 2).copy()
    X_val = X_val.transpose(0, 3, 1, 2).copy()
    X_test = X_test.transpose(0, 3, 1, 2).copy()

    return X_train, y_train, X_val, y_val, X_test, y_test

# Task 0: Implement backward pass for Fully Connected Layer
The `affine_forward` function in `cnn/layers.py` gives an implementation for the forward pass. Make sure you understand how it works. 

In [5]:
x = np.random.randn(4, 32)
w = np.random.randn(32, 2)
b = np.random.randn(2,)

out, _ = affine_forward(x, w, b)
print 'x shape: ', x.shape
print 'w shape: ', w.shape
print 'b shape: ', b.shape
print 'output shape: ' , out.shape

x shape:  (4, 32)
w shape:  (32, 2)
b shape:  (2,)
output shape:  (4, 2)


** Implement the affine_backward function in cnn/layers.py. **

When it is done, run the following code to check your backward pass with a numeric gradient check.

In [6]:
dout = np.random.randn(4, 2)

dx_num = eval_numerical_gradient_array(lambda x: affine_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_forward(x, w, b)[0], b, dout)

out, cache = affine_forward(x, w, b)
dx, dw, db = affine_backward(dout, cache)

# Your errors should be around 1e-9'
print 'Testing affine_backward function'
print 'dx error: ', rel_error(dx, dx_num)
print 'dw error: ', rel_error(dw, dw_num)
print 'db error: ', rel_error(db, db_num)

Testing affine_backward function
dx error:  2.49551962219e-09
dw error:  1.00019347521e-09
db error:  1.04012668057e-10


# Task 1: Implement backward pass for Convolution Layer
The core of a convolutional network is the convolution operation. The `conv_forward_naive` function in `cnn/layers.py` gives a forloop implementation for the forward pass. Make sure you understand how it works. 

In [7]:
x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3)

conv_param = {'stride': 2, 'pad': 1}
out, _ = conv_forward_naive(x, w, b, conv_param)
print 'x shape: ', x.shape
print 'w shape: ', w.shape
print 'b shape: ', b.shape
print 'output shape: ' , out.shape

x shape:  (2, 3, 4, 4)
w shape:  (3, 3, 4, 4)
b shape:  (3,)
output shape:  (2, 3, 2, 2)


**Implement the `conv_backward_naive` function in `cnn/layers.py`**. with forloops. 

When it is done, run the following code to check your backward pass with a numeric gradient check. 

In [8]:
x = np.random.randn(4, 3, 5, 5)
w = np.random.randn(2, 3, 3, 3)
b = np.random.randn(2,)
conv_param = {'stride': 2, 'pad': 1}
F, C, HH, WW = w.shape
N, _, H, W = x.shape 
dout = np.random.randn(4, 2, 1 + (W + 2 * conv_param['pad'] - WW) / conv_param['stride'], 1 + (W + 2 * conv_param['pad'] - WW) / conv_param['stride'])

dx_num = eval_numerical_gradient_array(lambda x: conv_forward_naive(x, w, b, conv_param)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv_forward_naive(x, w, b, conv_param)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: conv_forward_naive(x, w, b, conv_param)[0], b, dout)

out, cache = conv_forward_naive(x, w, b, conv_param)
dx, dw, db = conv_backward_naive(dout, cache)

# Your errors should be around 1e-9'
print 'Testing conv_backward_naive function'
print 'dx error: ', rel_error(dx, dx_num)
print 'dw error: ', rel_error(dw, dw_num)
print 'db error: ', rel_error(db, db_num)

Testing conv_backward_naive function
dx error:  6.39170218136e-10
dw error:  1.09929261977e-09
db error:  3.27628178591e-12


# Task 2: Implement backward pass for Max pooling
The max_pooling_forward_naive function in cnn/layers.py gives a forloop implementation for max pooling layer. Make sure you understand how it works.

In [9]:
x_shape = (2, 2, 2, 2)
x = np.linspace(-0.3, 0.4, num=np.prod(x_shape)).reshape(x_shape)
pool_param = {'pool_width': 2, 'pool_height': 2, 'stride': 2}

out, _ = max_pool_forward_naive(x, pool_param)

print 'x shape:' , x.shape
print 'output shape:' , out.shape
print 'x :' , x
print 'output: ' , out

x shape: (2, 2, 2, 2)
output shape: (2, 2, 1, 1)
x : [[[[-0.3        -0.25333333]
   [-0.20666667 -0.16      ]]

  [[-0.11333333 -0.06666667]
   [-0.02        0.02666667]]]


 [[[ 0.07333333  0.12      ]
   [ 0.16666667  0.21333333]]

  [[ 0.26        0.30666667]
   [ 0.35333333  0.4       ]]]]
output:  [[[[-0.16      ]]

  [[ 0.02666667]]]


 [[[ 0.21333333]]

  [[ 0.4       ]]]]


**Implement the `max_pool_backward_naive` function in the file `cnn/layers.py`** with forloop implementation here.

When it is done, run the following code to check your backward pass with a numeric gradient check. 

In [10]:
x = np.random.randn(3, 2, 8, 8)
dout = np.random.randn(3, 2, 4, 4)
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

dx_num = eval_numerical_gradient_array(lambda x: max_pool_forward_naive(x, pool_param)[0], x, dout)

out, cache = max_pool_forward_naive(x, pool_param)
dx = max_pool_backward_naive(dout, cache)

# Your error should be around 1e-12
print 'Testing max_pool_backward_naive function:'
print 'dx error: ', rel_error(dx, dx_num)

Testing max_pool_backward_naive function:
dx error:  3.27563083096e-12


# Task 3: Implement backward pass for Relu 

The relu_forward function in cnn/layers.py gives a forloop implementation for the Relu layer. Make sure you understand how it works.

In [11]:
x_shape = (1, 1, 2, 2)
x = np.linspace(-0.3, 0.3, num=np.prod(x_shape)).reshape(x_shape)
out, _ = relu_forward(x)

print 'x shape:' , x.shape
print 'output shape:' , out.shape
print 'x :' , x
print 'output: ' , out

x shape: (1, 1, 2, 2)
output shape: (1, 1, 2, 2)
x : [[[[-0.3 -0.1]
   [ 0.1  0.3]]]]
output:  [[[[-0.  -0. ]
   [ 0.1  0.3]]]]


**Implement the `relu_backward` function in `cnn/layers.py`**. 

When it is done, run the following code to check your backward pass with a numeric gradient check.

In [12]:
x = np.random.randn(1, 1, 2, 2)
dout = np.random.randn(1, 1, 2, 2)

dx_num = eval_numerical_gradient_array(lambda x: relu_forward(x)[0], x, dout)

out, cache = relu_forward(x)
dx = relu_backward(dout, cache)

print 'x shape:' , x.shape
print 'output shape:' , out.shape
print 'x :' , x
print 'output: ' , out
print 'dx: ' , dx

# Your error should be around 1e-12
print 'Testing relu_backward function:'
print 'dx error: ', rel_error(dx, dx_num)

x shape: (1, 1, 2, 2)
output shape: (1, 1, 2, 2)
x : [[[[-0.40420583 -0.57655132]
   [ 1.12101946 -0.33497201]]]]
output:  [[[[-0.         -0.        ]
   [ 1.12101946 -0.        ]]]]
dx:  [[[[-0.         -0.        ]
   [ 1.74034053  0.        ]]]]
Testing relu_backward function:
dx error:  3.27560157221e-12


# Task 4: Try Fast layer implementation
Making convolution and pooling layers fast can be challenging. We provided fast implementations of the forward and backward passes for convolution and pooling layers in the file `cnn/fast_layers.py`.

Run the follwing code to compare the speed of the naive implementation and the fast implementation. The fast convolution implementation depends on a Cython extension. **You need to compile it**:
* open a terminal with the "New" drop list in Jupyter (you can find it at the top right corner at http://localhost:8888).
* you run the following from the `cnn` directory.

```bash
python setup.py build_ext --inplace
```

**Understand how "conv_forward_fast" and "conv_backward_fast" are implmented in cnn/fast_layers.py.** The key for this faster implementation is to treat the convolutional layer as a fully connected layer. This is done by calling "im2col" to create a data matrix, where each column is the receptive field of a single convolution operation. 

Check this link for more details: https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/making_faster.html



In [13]:
x = np.random.randn(100, 3, 31, 31)
w = np.random.randn(25, 3, 3, 3)
b = np.random.randn(25,)
dout = np.random.randn(100, 25, 16, 16)
conv_param = {'stride': 2, 'pad': 1}

t0 = time()
out_naive, cache_naive = conv_forward_naive(x, w, b, conv_param)
t1 = time()
out_fast, cache_fast = conv_forward_fast(x, w, b, conv_param)
t2 = time()

print 'Testing conv_forward_fast:'
print 'Naive: %fs' % (t1 - t0)
print 'Fast: %fs' % (t2 - t1)
print 'Speedup: %fx' % ((t1 - t0) / (t2 - t1))
print 'Difference: ', rel_error(out_naive, out_fast)

t0 = time()
dx_naive, dw_naive, db_naive = conv_backward_naive(dout, cache_naive)
t1 = time()
dx_fast, dw_fast, db_fast = conv_backward_fast(dout, cache_fast)
t2 = time()

print '\nTesting conv_backward_fast:'
print 'Naive: %fs' % (t1 - t0)
print 'Fast: %fs' % (t2 - t1)
print 'Speedup: %fx' % ((t1 - t0) / (t2 - t1))
print 'dx difference: ', rel_error(dx_naive, dx_fast)
print 'dw difference: ', rel_error(dw_naive, dw_fast)
print 'db difference: ', rel_error(db_naive, db_fast)

Testing conv_forward_fast:
Naive: 2.821036s
Fast: 0.011230s
Speedup: 251.205511x
Difference:  3.37535377987e-11

Testing conv_backward_fast:
Naive: 3.846264s
Fast: 0.010529s
Speedup: 365.300507x
dx difference:  6.5734582088e-11
dw difference:  3.80895206143e-13
db difference:  1.27041950307e-14


In [16]:
x = np.random.randn(100, 3, 32, 32)
dout = np.random.randn(100, 3, 16, 16)
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

t0 = time()
out_naive, cache_naive = max_pool_forward_naive(x, pool_param)
t1 = time()
out_fast, cache_fast = max_pool_forward_fast(x, pool_param)
t2 = time()

print 'Testing pool_forward_fast:'
print 'Naive: %fs' % (t1 - t0)
print 'fast: %fs' % (t2 - t1)
print 'speedup: %fx' % ((t1 - t0) / (t2 - t1))
print 'difference: ', rel_error(out_naive, out_fast)

t0 = time()
dx_naive = max_pool_backward_naive(dout, cache_naive)
t1 = time()
dx_fast = max_pool_backward_fast(dout, cache_fast)
t2 = time()

print '\nTesting pool_backward_fast:'
print 'Naive: %fs' % (t1 - t0)
print 'speedup: %fx' % ((t1 - t0) / (t2 - t1))
print 'dx difference: ', rel_error(dx_naive, dx_fast)

Testing pool_forward_fast:
Naive: 0.183794s
fast: 0.003515s
speedup: 52.288408x
difference:  0.0

Testing pool_backward_fast:
Naive: 0.557820s
speedup: 57.560632x
dx difference:  0.0
